<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MARCH2026_NAT_DS_TUTORIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
random.seed(42)
print(random.random()) # Always 0.6394267984578837

0.6394267984578837


In [ ]:
!uv pip install "nvidia-nat[all]"

In [3]:
from google.colab import userdata
api_key = userdata.get('NVIDIA_API_KEY')
import os
os.environ['NVIDIA_API_KEY'] = api_key

api_key_TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = api_key_TAVILY_API_KEY

In [4]:
!nat --version

nat, version 1.4.1


## CASE 1: Basic ReAct Agent & Local Tools

In [5]:
from google.colab import userdata
import os
import yaml
import requests

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. DYNAMIC MODEL DISCOVERY
# This ensures we always use the current active version of the model.
headers = {"Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}"}
try:
    res = requests.get("https://integrate.api.nvidia.com/v1/models", headers=headers)
    all_models = [m['id'] for m in res.json().get('data', [])]
    # Selection logic for the latest distilled reasoning model
    ds_models = sorted([m for m in all_models if "deepseek-r1-distill" in m], reverse=True)
    active_model = ds_models[0] if ds_models else "deepseek-ai/deepseek-r1-distill-qwen-7b"
    print(f"✅ Active Model Identified: {active_model}")
except Exception:
    active_model = "deepseek-ai/deepseek-r1-distill-qwen-7b"

# 3. BUILD CONFIGURATION
# We inject the 'active_model' directly into the YAML dictionary.
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "openai",
            "model": active_model,
            "openai_api_base": "https://integrate.api.nvidia.com/v1",
            "openai_api_key": os.environ['NVIDIA_API_KEY'],
            "temperature": 0.01,
            "request_timeout": 600.0,
            "streaming": False
        }
    },
    "functions": {
        "current_time": { "_type": "nat.tool/current_datetime" },
        "web_search": { "_type": "nat.plugins.langchain.tools/tavily_internet_search" },
        "python_engine": {
            "_type": "nat.plugins.langchain.tools/code_generation",
            "llm_name": "reasoning_llm"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search", "python_engine"],
        "max_iterations": 10,
        "verbose": True,
        "handle_parsing_errors": True,
        "system_prompt": """
You are a Lead Aerospace Engineer AI.
CRITICAL: Do NOT use <think> tags. You MUST start every response with 'Thought:'.

You have access to:
{tools}
Available tool names: {tool_names}

MISSION:
1. Log current timestamp.
2. Search NASA Artemis II updates for cabin CO2 benchmarks.
3. Calculate fuel: (0.000001 * 50 * 86400 * 26000) / 3000.

Follow this format exactly:
Thought: [reasoning]
Action: [tool_name]
Action Input: {{"query": "..."}} or {{"unused": "dummy"}}
Observation: [result]
...
Final Answer: [Consolidated Report]
""".strip()
    }
}

with open("final_mission.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 4. EXECUTION
print(f"🚀 MISSION START - MODEL INJECTED: {active_model}")
!nat run --config_file final_mission.yml --input "Conduct the ART-II diagnostic and calculate station-keeping for 50m."

✅ Active Model Identified: deepseek-ai/deepseek-r1-distill-qwen-7b
🚀 MISSION START - MODEL INJECTED: deepseek-ai/deepseek-r1-distill-qwen-7b
2026-02-15 20:41:15 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_mission.yml'
2026-02-15 20:41:15 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function code_generation_tool by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.
2026-02-15 20:41:15 - INFO     - nat.plugins.langchain.tools.code_generation_tool:43 - Initializing code generation tool
Getting tool LLM from config
2026-02-15 20:41:17 - INFO     - nat.plugins.langchain.tools.code_generation_tool:54 - Filling tool's prompt variable from config
2026-02-15 20:41:17 - INFO     - nat.plugins.langchain.tools.code_generation_tool:57 - Initialized code generation tool
2026-02-15 20:4

## CASE 2: Template Validation & Literal Escaping

In [1]:
# ============================================================================
# DEEPSEEK-R1 CASE 2 - STABILIZED RECTIFIED PROMPT
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
active_model = "deepseek-ai/deepseek-r1-distill-qwen-7b"

config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "openai",
            "model": active_model,
            "openai_api_base": "https://integrate.api.nvidia.com/v1",
            "openai_api_key": os.environ['NVIDIA_API_KEY'],
            "temperature": 0.01,
            "request_timeout": 600.0,
            "streaming": False
        }
    },
    "functions": {
        "current_time": { "_type": "nat.tool/current_datetime" },
        "web_search": { "_type": "nat.plugins.langchain.tools/tavily_internet_search" }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search"],
        "max_iterations": 10,
        "verbose": True,
        "handle_parsing_errors": True,
        "system_prompt": """
Analyze mission telemetry and cross-reference with NASA ECLSS standards.
CRITICAL: You MUST start every response with 'Thought:'. Do NOT use <think> tags.

You have access to the following tools:
{tools}

Follow this format exactly:
Thought: I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: {{"unused": "dummy"}} or {{"query": "search term"}}
Observation: the result of the action
...
Final Answer: [Your diagnosis]

NASA REFERENCE DATA:
- CO2 Standard: < 3900 ppm
- Scrubber Power Goal: <= 500 Watts
""".strip()
    }
}

with open("stable_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
mission_input = "1. Get the current timestamp. 2. Diagnose 142W / 5.2 ppm telemetry based on the provided NASA standards."
!nat run --config_file stable_workflow.yml --input "{mission_input}"

2026-02-15 20:50:01 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'stable_workflow.yml'
2026-02-15 20:50:05.106248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 20:50:05.111707: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 20:50:05.125829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771188605.150130    3609 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771188605.157055    3609 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771188605.175

## CASE 3: Tavily Web Search Integration

In [1]:
# ============================================================================
# DEEPSEEK-R1 CASE 3 - FINAL STABILIZED (NO-GUESSING FIX)
# ============================================================================
import os, yaml, requests
from google.colab import userdata

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. DYNAMIC MODEL DISCOVERY
headers = {"Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}"}
try:
    res = requests.get("https://integrate.api.nvidia.com/v1/models", headers=headers)
    ds_models = sorted([m['id'] for m in res.json().get('data', []) if "deepseek-r1-distill" in m['id']], reverse=True)
    active_model = ds_models[0] if ds_models else "deepseek-ai/deepseek-r1-distill-qwen-7b"
except Exception:
    active_model = "deepseek-ai/deepseek-r1-distill-qwen-7b"

# 3. UPDATED CONFIGURATION
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "openai",
            "model": active_model,
            "openai_api_base": "https://integrate.api.nvidia.com/v1",
            "openai_api_key": os.environ['NVIDIA_API_KEY'],
            "temperature": 0.01,
            "request_timeout": 600.0, # FIXED: Gives model 10 mins to think
            "streaming": False        # FIXED: Buffers full response to avoid chunked errors
        }
    },
    "functions": {
        "web_search": { "_type": "nat.plugins.langchain.tools/tavily_internet_search" }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["web_search"],
        "max_iterations": 5,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 10,
        "system_prompt": """
Analyze mission telemetry.
CRITICAL: Do NOT use <think> tags. You MUST start every response with 'Thought:'.

PROHIBITION: You are FORBIDDEN from guessing NASA benchmarks.
You MUST use 'web_search' to verify the Artemis II CO2 ppm limit before providing a final answer.

You have access to: {tools}
Available tool names: {tool_names}

Follow this format exactly:
Thought: I need to verify the official NASA CO2 limit for Artemis II.
Action: web_search
Action Input: {{"question": "NASA Standard 3001 Artemis II CO2 limits"}}
Observation: [result]
...
Final Answer: [Detailed Diagnosis]
""".strip()
    }
}

with open("tavily_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 4. EXECUTION
mission_input = "Search for official NASA Artemis II CO2 ppm limits and compare with 5.2 ppm telemetry."

print(f"🚀 CASE 3 RE-INITIATING - MODEL: {active_model}")
!nat run --config_file tavily_workflow.yml --input "{mission_input}"

🚀 CASE 3 RE-INITIATING - MODEL: deepseek-ai/deepseek-r1-distill-qwen-7b
2026-02-15 21:32:20 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'tavily_workflow.yml'
2026-02-15 21:32:24.336207: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 21:32:24.343411: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 21:32:24.366012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771191144.406031   14070 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771191144.416311   14070 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugi

## CASE 4: Stable Multi-Tool Configuration

In [1]:
# ============================================================================
# DEEPSEEK-R1 MISSION - CASE 4 FINAL STABILIZED (MULTI-TOOL SUCCESS)
# ============================================================================
from google.colab import userdata
import os, yaml, requests

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. DYNAMIC MODEL DISCOVERY
headers = {"Authorization": f"Bearer {os.environ['NVIDIA_API_KEY']}"}
try:
    res = requests.get("https://integrate.api.nvidia.com/v1/models", headers=headers)
    ds_models = sorted([m['id'] for m in res.json().get('data', []) if "deepseek-r1-distill" in m['id']], reverse=True)
    active_model = ds_models[0] if ds_models else "deepseek-ai/deepseek-r1-distill-qwen-7b"
except Exception:
    active_model = "deepseek-ai/deepseek-r1-distill-qwen-7b"

# 3. UPDATED WORKFLOW CONFIGURATION (NAT 1.4.1 COMPATIBLE)
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "openai",
            "model": active_model,
            "openai_api_base": "https://integrate.api.nvidia.com/v1",
            "openai_api_key": os.environ['NVIDIA_API_KEY'],
            "temperature": 0.01,
            "request_timeout": 600.0, # FIXED: Prevents timeout during dual-tool reasoning
            "streaming": False        # FIXED: Prevents "incomplete chunked read"
        }
    },
    "functions": {
        "current_time": { "_type": "nat.tool/current_datetime" },
        "web_search": { "_type": "nat.plugins.langchain.tools/tavily_internet_search" }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search"],
        "max_iterations": 10,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 10,
        "system_prompt": """
You are a mission diagnostic specialist and historian.
CRITICAL: Do NOT use <think> tags. You MUST start every response with 'Thought:'.

You have access to: {tools}
Available tools: {tool_names}

Analyze telemetry: 142W / 5.2 ppm.

MISSION SEQUENCE RULE (H2E GOVERNANCE):
1. Call 'current_time' exactly ONCE to establish the log timestamp.
2. Immediately call 'web_search' to find official NASA Artemis II CO2 limits.
3. You are FORBIDDEN from calling the same tool twice in a row.
4. If you have the timestamp and the limits, provide the Final Answer immediately.

STRICT TOOL RULES:
- For 'web_search', you MUST use the key 'question'.
- For 'current_time', you MUST use Action Input: {{"unused": "dummy"}}

Follow this format:
Thought: I need to [reason]
Action: [tool_name]
Action Input: [JSON object with correct keys]
Observation: [result]
...
Final Answer: [Consolidated Report]
""".strip()
    }
}

with open("stable_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 4. EXECUTION
mission_input = "Execute the collaborative diagnostic of Mission ART-II (Telemetry: 142W / 5.2 ppm)."

print(f"🚀 CASE 4 FINAL RUN - MODEL: {active_model}")
print("-" * 70)

!nat run --config_file stable_workflow.yml --input "{mission_input}"

🚀 CASE 4 FINAL RUN - MODEL: deepseek-ai/deepseek-r1-distill-qwen-7b
----------------------------------------------------------------------
2026-02-15 21:36:04 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'stable_workflow.yml'
2026-02-15 21:36:08.547576: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 21:36:08.554972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-15 21:36:08.577071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771191368.614994   15002 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771191368.626258   15002 cuda_blas.cc:1407] Unable t

## CASE 5: One-Shot Finalization & Code Execution

In [1]:
# ============================================================================
# DEEPSEEK-R1 MISSION - FINAL STABILIZED CASE 5
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# Use the distilled model optimized for the NVIDIA NIM endpoint
active_model = "deepseek-ai/deepseek-r1-distill-qwen-7b"

# 2. DEFINE THE WORKFLOW
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "openai", # Use 'openai' for broad parameter compatibility
            "model": active_model,
            "openai_api_base": "https://integrate.api.nvidia.com/v1",
            "openai_api_key": os.environ['NVIDIA_API_KEY'],
            "temperature": 0.01,
            # FIXED: 600s timeout prevents the "peer closed connection" error
            "request_timeout": 600.0,
            # FIXED: False prevents "incomplete chunked read" from mid-stream failures
            "streaming": False,
            # We omit max_tokens to prevent the 400 'extra_forbidden' error
        }
    },
    "functions": {
        "current_time": { "_type": "nat.tool/current_datetime" },
        "web_search": { "_type": "nat.plugins.langchain.tools/tavily_internet_search" },
        "python_engine": {
            "_type": "nat.plugins.langchain.tools/code_generation",
            "llm_name": "reasoning_llm"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search", "python_engine"],
        "max_iterations": 6,
        "verbose": True,
        "handle_parsing_errors": True,
        "system_prompt": """
You are a Lead Aerospace Engineer AI.
CRITICAL: Do NOT use <think> tags. You MUST start every response with 'Thought:'.
After using python_engine once, you MUST stop and provide the Final Answer immediately.

You have access to: {tools}
Available tools: {tool_names}

MISSION:
1. Log current timestamp using Action Input: {{"unused": "dummy"}}.
2. Search NASA Artemis II updates for cabin CO2 benchmarks.
3. Use python_engine to calculate fuel: (0.000001 * 50 * 86400 * 26000) / 3000

Follow this format exactly:
Thought: [reasoning]
Action: [tool_name]
Action Input: {{"query": "..."}} or {{"unused": "dummy"}}
Observation: [result]
...
Final Answer: [Consolidated Report]
""".strip()
    }
}

with open("final_mission.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
print(f"🚀 MISSION START - STABILITY PATCHES APPLIED - MODEL: {active_model}")
print("-" * 70)

!nat run --config_file final_mission.yml --input "Conduct the ART-II diagnostic and calculate station-keeping for 50m."

🚀 MISSION START - STABILITY PATCHES APPLIED - MODEL: deepseek-ai/deepseek-r1-distill-qwen-7b
----------------------------------------------------------------------
2026-02-15 21:37:29 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_mission.yml'
2026-02-15 21:37:29 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function code_generation_tool by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.
2026-02-15 21:37:29 - INFO     - nat.plugins.langchain.tools.code_generation_tool:43 - Initializing code generation tool
Getting tool LLM from config
2026-02-15 21:37:30 - INFO     - nat.plugins.langchain.tools.code_generation_tool:54 - Filling tool's prompt variable from config
2026-02-15 21:37:30 - INFO     - nat.plugins.langchain.tools.code_generation_tool:57 - Initialized code generati